In [78]:
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import torch

# Importing libraries
import copy
import os

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset
from typing import Tuple, List
from torch.optim import Optimizer

from nltk.corpus import words
import nltk

import numpy as np

In [79]:
!pip install emoji

In [80]:
import emoji

In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
import os

BASE_PATH = 'drive/MyDrive/NLP_project/'
# BASE_PATH = './'
if not os.path.exists(BASE_PATH):
    raise ValueError('path does not exist')

In [83]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [84]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


# Load dataset

In [85]:
df_train = pd.read_csv(BASE_PATH + 'datasets/EmoTrain.csv')
df_test = pd.read_csv(BASE_PATH + 'datasets/EmoVal.csv')

In [86]:
df_train = df_train.drop(axis=1, columns=['Unnamed: 0'])
df_test = df_test.drop(axis=1, columns=['Unnamed: 0'])
df_train

,text,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
0,Is there some scripture you could quote me? I'...,1,0,0,0,0,1,0,0,0,0
1,Good. Now we just need people to dislike commi...,1,0,0,0,0,0,0,0,0,0
2,This was driving me NUTS!,0,1,0,0,0,0,0,0,0,0
3,Thank you for your advice!,0,0,0,0,0,0,0,0,0,1
4,Some do. Some don't. Blanket generalizations a...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
190097,They needed to insert the phrase “over mainten...,0,0,0,0,1,0,0,0,0,0
190098,Back in the seventies and eighties we all did ...,1,0,0,0,0,0,0,0,0,0
190099,"6lbs is a lap dog, if someone shoots that caus...",0,0,0,0,0,0,0,1,0,0
190100,This gets much worse on the 2nd loop.,0,0,0,0,1,0,0,0,0,0


In [87]:
# step = 5_000
df_train = df_train[:130_000]
df_test = df_test[:130_000]

# Pre processing

## Description of data

In [88]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   text         130000 non-null  object
 1   praise       130000 non-null  int64 
 2   amusement    130000 non-null  int64 
 3   anger        130000 non-null  int64 
 4   disapproval  130000 non-null  int64 
 5   confusion    130000 non-null  int64 
 6   interest     130000 non-null  int64 
 7   sadness      130000 non-null  int64 
 8   fear         130000 non-null  int64 
 9   joy          130000 non-null  int64 
 10  love         130000 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 10.9+ MB


In [89]:
print(df_train.isnull().sum())

text           0
praise         0
amusement      0
anger          0
disapproval    0
confusion      0
interest       0
sadness        0
fear           0
joy            0
love           0
dtype: int64


In [90]:
def split_features_labels(df) -> Tuple[pd.Series, pd.DataFrame]:
    x = df['text']
    y = df.drop(axis=1, columns=['text'])
    return x, y


x_train, y_train = split_features_labels(df_train)
x_test, y_test = split_features_labels(df_test)

In [91]:
x_train

,text
0,Is there some scripture you could quote me? I'...
1,Good. Now we just need people to dislike commi...
2,This was driving me NUTS!
3,Thank you for your advice!
4,Some do. Some don't. Blanket generalizations a...
...,...
129995,Because lowest price technically acceptable is...
129996,I am really sad because my innocent nieces are...
129997,Wear you're ear pro kids. Styro plugs are chea...
129998,"She's still looking for her [NAME], maybe? :)"


In [92]:
y_train

,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
0,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1
4,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
129995,0,0,0,0,1,0,0,0,0,0
129996,0,0,0,0,0,0,1,0,0,0
129997,0,0,0,0,0,0,0,0,0,1
129998,0,0,0,0,1,0,0,0,0,0


## Data cleaning

Lower case all the words

In [93]:
def lower_case(x):
    x = x.str.lower()
    return x

x_train = lower_case(x_train)
x_test = lower_case(x_test)

In [94]:
x_train

,text
0,is there some scripture you could quote me? i'...
1,good. now we just need people to dislike commi...
2,this was driving me nuts!
3,thank you for your advice!
4,some do. some don't. blanket generalizations a...
...,...
129995,because lowest price technically acceptable is...
129996,i am really sad because my innocent nieces are...
129997,wear you're ear pro kids. styro plugs are chea...
129998,"she's still looking for her [name], maybe? :)"


Convert to tokens

In [95]:
x_train = x_train.to_list()
x_test = x_test.to_list()

In [96]:
print(len(x_train))
print(x_train[:5])

130000
["is there some scripture you could quote me? i'd like to read up on it just to be sure for myself", 'good. now we just need people to dislike commies more than they do now', 'this was driving me nuts!', 'thank you for your advice!', "some do. some don't. blanket generalizations are almost always false and unhelpful."]


In [97]:
def list_of_words(x):
    x = list(map(lambda i: i.split(), x))
    return x

x_train = list_of_words(x_train)
x_test = list_of_words(x_test)

Then we have **list** of **list** of **tokens**

In [98]:
x_train

[['is',
  'there',
  'some',
  'scripture',
  'you',
  'could',
  'quote',
  'me?',
  "i'd",
  'like',
  'to',
  'read',
  'up',
  'on',
  'it',
  'just',
  'to',
  'be',
  'sure',
  'for',
  'myself'],
 ['good.',
  'now',
  'we',
  'just',
  'need',
  'people',
  'to',
  'dislike',
  'commies',
  'more',
  'than',
  'they',
  'do',
  'now'],
 ['this', 'was', 'driving', 'me', 'nuts!'],
 ['thank', 'you', 'for', 'your', 'advice!'],
 ['some',
  'do.',
  'some',
  "don't.",
  'blanket',
  'generalizations',
  'are',
  'almost',
  'always',
  'false',
  'and',
  'unhelpful.'],
 ['those', 'are', 'separate', 'issues', 'from', 'the', 'sample', 'size.'],
 ['my',
  'fur',
  'son',
  'was',
  'the',
  'first',
  'i',
  'came',
  'out',
  'to.',
  'most',
  'important',
  'also'],
 ['really',
  'appreciate',
  'this',
  'post',
  'and',
  'article.',
  'i',
  'just',
  'subscribed',
  'and',
  'between',
  'this',
  'and',
  'the',
  'sidebar',
  "i'm",
  'feeling',
  'pumped',
  'about',
  'start

What kind of **characters** are there in the dataset?

In [99]:
characters = {'isalnum': 0}
for text in x_train:
    for word in text:
        for c in word:
            if c.isalnum():
                characters['isalnum'] += 1
            elif c in characters:
                characters[c] += 1
            else:
                characters[c] = 0

In [100]:
print(len(characters))
keys = list(characters.keys())
keys.sort()
print(keys)
characters

461
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'isalnum', '{', '|', '}', '~', '¡', '£', '¦', '©', '«', '¬', '¯', '°', '´', '·', '»', '̕', '̖', '̗', '̘', '̙', '̜', '̝', '̞', '̟', '̠', '̢', '̣', '̤', '̥', '̦', '̧', '̨', '̪', '̫', '̬', '̭', '̮', '̯', '̰', '̱', '̲', '̳', '̶', '̷', '̹', '̺', '̻', '̼', 'ͅ', '͈', '͍', '͎', '͏', '͓', '͔', '͕', '͖', '͘', '͙', '͚', '͜', '͝', '͞', '͟', '͠', '͡', '͢', '׳', '\u200d', '–', '—', '‘', '’', '“', '”', '„', '•', '…', '\u202a', '\u202c', '‽', '€', '™', '√', '≈', '≠', '▀', '▫', '☁', '☂', '☆', '☕', '☝', '☠', '☹', '☺', '♀', '♂', '♡', '♥', '♪', '♫', '♭', '♾', '♿', '⚔', '⚰', '⛏', '⛑', '✊', '✋', '✌', '✨', '❄', '❣', '❤', '⠀', '⠁', '⠃', '⠄', '⠇', '⠈', '⠉', '⠊', '⠋', '⠏', '⠓', '⠘', '⠙', '⠚', '⠛', '⠞', '⠟', '⠢', '⠦', '⠫', '⠳', '⠴', '⠶', '⠸', '⠹', '⠻', '⠾', '⠿', '⡀', '⡄', '⡆', '⡇', '⡏', '⡜', '⡟', '⡶', '⡷', '⡼', '⡾', '⡿', '⢀', '⢠', '⢣', '⢤', '⢧', '⢰', '⢸', '⢹', '⢻', '⢿'

{'isalnum': 7069081,
 '?': 16000,
 "'": 45011,
 '.': 141856,
 '!': 25668,
 '’': 20337,
 '[': 23270,
 ']': 23259,
 ',': 43122,
 '"': 7354,
 ':': 4666,
 ')': 3525,
 '-': 5959,
 '💩': 2,
 '/': 3788,
 '*': 5889,
 '(': 2626,
 '^': 917,
 '>': 2063,
 '~': 679,
 '”': 1503,
 '😒': 15,
 '£': 30,
 '“': 1510,
 '🙄': 62,
 '🍻': 6,
 '—': 71,
 '😂': 776,
 '=': 285,
 '$': 515,
 '😍': 75,
 '😪': 9,
 ';': 611,
 '&': 242,
 '😭': 147,
 '❤': 232,
 '️': 475,
 '%': 635,
 '͡': 31,
 '͜': 19,
 '🅱': 22,
 '😔': 29,
 '+': 349,
 '🤕': 6,
 '@': 88,
 '🤣': 227,
 '#': 520,
 '🙌': 33,
 '😇': 7,
 '🎶': 33,
 '💃': 21,
 '<': 341,
 '\u200d': 149,
 '♀': 93,
 '☺': 45,
 '😢': 56,
 '😦': 1,
 '😜': 9,
 '♡': 2,
 '_': 432,
 '‘': 212,
 '💪': 13,
 '🤔': 108,
 '😣': 7,
 '😊': 61,
 '🐢': 2,
 '😄': 21,
 '😐': 21,
 '😤': 187,
 '😞': 8,
 '¦': 2,
 '¯': 55,
 '😱': 19,
 '👍': 87,
 '🍑': 1,
 '😹': 1,
 '👌': 58,
 '🏾': 8,
 '😕': 7,
 '♂': 54,
 '🏻': 92,
 '–': 7,
 '💕': 30,
 '❣': 8,
 '😎': 52,
 '😡': 30,
 '🤬': 7,
 '😴': 3,
 '€': 19,
 '👏': 124,
 '™': 20,
 '🤢': 30,
 '🕺': 2,
 '🙃': 29,

- Remove non-semantic characters
- Convert `?` and `!` and `emojis` to tokens

In [101]:
def split_special_chars(word: str):
    if word.isalnum():
        return word, []
    specials = list(filter(lambda c: c == '!' or c == '?', word))
    return ''.join(filter(lambda c: c.isalnum(), word)), specials

def clean_tokens(x):
    for text in tqdm(x):
        specials = []
        for i, word in enumerate(text):
            text[i], special_word = split_special_chars(word)
            specials.extend(special_word)
        text.extend(specials)

clean_tokens(x_train)
clean_tokens(x_test)

  0%|          | 0/130000 [00:00<?, ?it/s]

  0%|          | 0/10562 [00:00<?, ?it/s]

In [102]:
x_train

[['is',
  'there',
  'some',
  'scripture',
  'you',
  'could',
  'quote',
  'me',
  'id',
  'like',
  'to',
  'read',
  'up',
  'on',
  'it',
  'just',
  'to',
  'be',
  'sure',
  'for',
  'myself',
  '?'],
 ['good',
  'now',
  'we',
  'just',
  'need',
  'people',
  'to',
  'dislike',
  'commies',
  'more',
  'than',
  'they',
  'do',
  'now'],
 ['this', 'was', 'driving', 'me', 'nuts', '!'],
 ['thank', 'you', 'for', 'your', 'advice', '!'],
 ['some',
  'do',
  'some',
  'dont',
  'blanket',
  'generalizations',
  'are',
  'almost',
  'always',
  'false',
  'and',
  'unhelpful'],
 ['those', 'are', 'separate', 'issues', 'from', 'the', 'sample', 'size'],
 ['my',
  'fur',
  'son',
  'was',
  'the',
  'first',
  'i',
  'came',
  'out',
  'to',
  'most',
  'important',
  'also'],
 ['really',
  'appreciate',
  'this',
  'post',
  'and',
  'article',
  'i',
  'just',
  'subscribed',
  'and',
  'between',
  'this',
  'and',
  'the',
  'sidebar',
  'im',
  'feeling',
  'pumped',
  'about',
  's

Remove stop-words

In [103]:
print(len(ENGLISH_STOP_WORDS))
print(ENGLISH_STOP_WORDS)

318
frozenset({'forty', 'namely', 'via', 'among', 'eleven', 'must', 'because', 'cry', 'without', 'very', 'less', 'anyway', 'least', 'would', 'the', 'a', 'yourselves', 'serious', 'twenty', 'thereby', 'another', 'what', 'and', 'ours', 'nor', 'themselves', 'cant', 'call', 'afterwards', 'five', 'everyone', 'becomes', 'we', 'with', 'be', 'still', 'through', 'in', 'more', 'done', 'beyond', 'latter', 'becoming', 'as', 'both', 'she', 'ever', 'amongst', 'anyhow', 'alone', 'thereafter', 'give', 'anywhere', 'further', 'one', 'twelve', 'not', 'itself', 'an', 'either', 'you', 'meanwhile', 'go', 'me', 'well', 'name', 'seem', 'always', 'none', 'many', 'hasnt', 'whoever', 'they', 'most', 'ie', 'seems', 'can', 'is', 'whereby', 'into', 'upon', 'hereby', 'inc', 'last', 'has', 'system', 'everything', 'describe', 'couldnt', 'were', 'other', 'amoungst', 'cannot', 'after', 'amount', 'everywhere', 'nevertheless', 'somehow', 'except', 'behind', 'myself', 'two', 'hence', 'such', 'have', 'above', 'i', 'somewhere

In [104]:
def remove_stop_words(x):
    x = list(map(lambda text: list(filter(lambda word: word not in ENGLISH_STOP_WORDS, text)), x))
    return x

x_train = remove_stop_words(x_train)
x_test = remove_stop_words(x_test)

# Vectorize the dataset

An example of vectorization

In [105]:
df = pd.DataFrame(dtype=bool)
for i, text in enumerate(tqdm(x_train[:5])):
    for word in text:
        df.loc[i, word] = True
df.fillna(False, inplace=True)

  0%|          | 0/5 [00:00<?, ?it/s]

<ipython-input-105-c6cb43738d2c>:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(False, inplace=True)


In [106]:
df

,scripture,quote,id,like,read,just,sure,?,good,need,...,driving,nuts,!,thank,advice,dont,blanket,generalizations,false,unhelpful
0,True,True,True,True,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,True,True


In [107]:
top_words = {}
for text in x_train:
    for word in text:
            if word in top_words:
                top_words[word] += 1
            else:
                top_words[word] = 1

top_words_ordered = sorted(top_words.items(), key=lambda kv: kv[1])
print(top_words_ordered)

[('frend', 1), ('beforerevolution', 1), ('actally', 1), ('representations', 1), ('cools', 1), ('looping', 1), ('1911', 1), ('rummages', 1), ('foals', 1), ('50100', 1), ('histories', 1), ('rwoosh', 1), ('backstage', 1), ('blower', 1), ('welcher', 1), ('refrigeration', 1), ('deflating', 1), ('themwait', 1), ('balancing', 1), ('korver', 1), ('spitballing', 1), ('tpss', 1), ('antifascist', 1), ('itation', 1), ('eeded', 1), ('jumpshot', 1), ('tweezer', 1), ('gushed', 1), ('anxieties', 1), ('molon', 1), ('labe', 1), ('squanch', 1), ('biblethump', 1), ('pita', 1), ('ujannist', 1), ('bejebuzz', 1), ('skynet', 1), ('ubeelzebambi', 1), ('stupotity', 1), ('danbo', 1), ('congratulation', 1), ('qualifiers', 1), ('snowbirds', 1), ('iwreck', 1), ('respectability', 1), ('soar', 1), ('commemorating', 1), ('anuses', 1), ('instructive', 1), ('upvoters', 1), ('whatss', 1), ('kotal', 1), ('treading', 1), ('rredditsmuseumoffilth', 1), ('2lbs', 1), ('frustrate', 1), ('pussycat', 1), ('seventh', 1), ('sadbut'

In [108]:
print(list(map(lambda x: x[1], top_words_ordered)).index(1))
print(list(map(lambda x: x[1], top_words_ordered)).index(2))
print(list(map(lambda x: x[1], top_words_ordered)).index(3))
print(list(map(lambda x: x[1], top_words_ordered)).index(4))
print(list(map(lambda x: x[1], top_words_ordered)).index(5))

0
3549
10316
15717
18637


In [109]:
word_list = list(map(lambda x: x[0], top_words_ordered[list(map(lambda x: x[1], top_words_ordered)).index(3):]))
print(len(word_list))

21844


Create an index for every unique word

In [110]:
tensor_index = {}
for i, word in enumerate(tqdm(word_list)):
    tensor_index[word] = i

  0%|          | 0/21844 [00:00<?, ?it/s]

In [111]:
print(len(tensor_index))

21844


Create a tensor

In [112]:
def create_x_tensor(x):
    x_tensor = torch.zeros(len(x), len(tensor_index), dtype=torch.float16)
    for i, text in enumerate(tqdm(x)):
        for word in text:
            if word in tensor_index:
                x_tensor[i, tensor_index[word]] += 1
    return x_tensor

In [113]:
# x_train
# last_index = 0
# for i in range(0, len(x_train), step):
#     last_index = i
#     torch.save(create_x_tensor(x_train[i:i+step]), BASE_PATH + f'tensors/x_train_tensor{i//step}.pt')
# torch.save(create_x_tensor(x_train[last_index:]), BASE_PATH + f'tensors/x_train_tensor{last_index//step + 1}.pt')

In [114]:
torch.save(create_x_tensor(x_train), BASE_PATH + 'tensors/x_train_tensor.pt')
torch.save(create_x_tensor(x_test), BASE_PATH + 'tensors/x_test_tensor.pt')

  0%|          | 0/130000 [00:00<?, ?it/s]

  0%|          | 0/10562 [00:00<?, ?it/s]

Convert labels to tensor

In [115]:
y_train.describe()

,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
count,130000.000000,130000.000000,130000.000000,130000.000000,130000.000000,130000.000000,130000.000000,130000.000000,130000.000000,130000.000000
mean,0.157331,0.069885,0.095408,0.077331,0.334292,0.063723,0.080862,0.021354,0.127838,0.117338
std,0.364114,0.254953,0.293778,0.267117,0.471744,0.244260,0.272623,0.144561,0.333911,0.321824
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [116]:
def create_y_tensor(y):
    y_tensor = torch.tensor(y.values, dtype=torch.float16)
    return y_tensor

In [117]:
# y_train
# last_index = 0
# for i in range(0, len(y_train), step):
#     last_index = i
#     torch.save(create_y_tensor(y_train[i:i+step]), BASE_PATH + f'tensors/y_train_tensor{i//step}.pt')
# torch.save(create_y_tensor(y_train[last_index:]), BASE_PATH + f'tensors/y_train_tensor{last_index//step + 1}.pt')

In [118]:
torch.save(create_y_tensor(y_train), BASE_PATH + 'tensors/y_train_tensor.pt')
torch.save(create_y_tensor(y_test), BASE_PATH + 'tensors/y_test_tensor.pt')